In [84]:
import pandas as pd
import numpy as np
from config import *
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
df_event = pd.read_parquet(PREPROCESSED_DIR / "anti_saccade_preprocessed_from_r.pq")
df_sample = (pd.read_parquet(RAW_DIR / 'ANTI_SACCADE_SAMPLES.pq')
 .sort_values(["experiment", "participant_id", "trial_id","time"])
)

def rename_columns(df):
    """Renames columns by joining multi-level column names with different delimiters."""
    # Iterate over all column names
    df.columns = [f"{col[0]}" if col[1] == '' else f"{col[0]}_{col[1]}" for col in df.columns.values]
    return df

# Event features

In [ ]:
def get_trial_correctness_df(df:pd.DataFrame) -> pd.DataFrame:
    df_trials = (df
        .query('stimulus_active == True')
        .sort_values(by=["participant_id", "trial_id", "eye","stand_time"])
        .assign(stimulus_time = lambda x: np.select([x.event == "FIXPOINT", x.event != "FIXPOINT"], [x.stand_time, None]))
        .assign(stimulus_time = lambda x: x["stimulus_time"].ffill())
        .assign(saccade_direction = lambda x: np.select([(x["event"] == 'ESACC') & (np.abs(x["sacc_end_x"] - x["sacc_start_x"]) < 50),
                                                        (x["event"] == 'ESACC') & (x["sacc_end_x"] > x["sacc_start_x"]),
                                                        (x["event"] == 'ESACC') & (x["sacc_end_x"] < x["sacc_start_x"])],
                                                        ['no_direction',"right", "left"], default=None))
        .assign(saccade_end_area = lambda x: np.select([(x["event"] == 'ESACC') & ( 840 < x["sacc_end_x"]) & (x["sacc_end_x"] < 1080),
                                                        (x["event"] == 'ESACC') & (1080 <= x["sacc_end_x"]),
                                                        (x["event"] == 'ESACC') & (x["sacc_end_x"] <= 840)],
                                                    ['middle',"right", "left"], default=None))
        .assign(is_saccade_correct = lambda x: np.select([(x["saccade_direction"] == 'no_direction')
                                                        , (x["saccade_end_area"] == 'middle')
                                                        , (x["saccade_direction"] == x["saccade_end_area"]) & (x["saccade_direction"] != x["stimulus_side"]) & (x["saccade_end_area"] != x["stimulus_side"])
                                                        , (x["saccade_direction"] == x["saccade_end_area"]) & (x["saccade_direction"] == x["stimulus_side"]) & (x["saccade_end_area"] == x["stimulus_side"])
                                                        ],
                                                            [None, None, True, False], default=None)) 
        .assign(is_trial_correct = lambda x: (
                x.sort_values(by=["participant_id", "trial_id", "stand_time"])
                .groupby(["participant_id", "trial_id"])["is_saccade_correct"]
                .transform(lambda group: (
                    True if not group.dropna().empty and group.dropna().iloc[0] == True else
                    False if not group.dropna().empty and group.dropna().iloc[0] == False else
                    None
                ))
            ))
    )
    return df_trials

def get_n_correct_trials_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns pd.Dataframe with columns ['experiment', 'participant_id', 'n_correct_trials']
    """
    
    feature_df = (df
     .pipe(get_trial_correctness_df)
     .groupby(["experiment","participant_id", "trial_id"])
     .agg(is_trial_correct = ('is_trial_correct', 'min')) 
     .reset_index()
     .groupby(["experiment", "participant_id"])
     .agg(n_correct_trials = ('is_trial_correct', 'sum'))
     .reset_index()
    [["experiment", "participant_id", "n_correct_trials"]]
    )
    
    return feature_df


def get_prop_trials_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns pd.Dataframe with columns ['experiment', 'participant_id', 'prop_correct_trials']
    """
    
    feature_df = (df
     .pipe(get_trial_correctness_df)
     .groupby(["experiment","participant_id", "trial_id"])
     .agg(is_trial_correct = ('is_trial_correct', 'min')) 
     .reset_index()
     .groupby(["experiment", "participant_id"])
     .agg(n_correct_trials = ('is_trial_correct', 'sum'),
          n_trials = ('is_trial_correct', 'count'))
     .reset_index()
     .assign(prop_correct_trials = lambda x: x["n_correct_trials"] / x["n_trials"])
     [["experiment", "participant_id", "prop_correct_trials"]]
    )
    
    return feature_df

def get_reaction_time_feature(df: pd.DataFrame) -> pd.DataFrame:
    return (df
        .pipe(get_trial_correctness_df)
        .assign(is_saccade_correct = lambda x: np.select([ (x["is_saccade_correct"] == True) ], [True], default=None))
        .groupby(["experiment","participant_id", "trial_id"])
        # TODO: Create the correct logic here. 
    )
    
get_reaction_time_feature(df_event)
     

/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_32428/116951766.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .assign(stimulus_time = lambda x: x["stimulus_time"].ffill())
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_32428/116951766.py:77: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.dropna(how="all").head(1).iloc[0].dropna())


,experiment,participant_id,trial_id,level_3,0
0,ANTI_SACCADE,103,0.0,experiment,ANTI_SACCADE
1,ANTI_SACCADE,103,0.0,participant_id,103
2,ANTI_SACCADE,103,0.0,trial_id,0.0
3,ANTI_SACCADE,103,0.0,stand_time,698.331778
4,ANTI_SACCADE,103,0.0,eye,L
...,...,...,...,...,...
39143,ANTI_SACCADE,96,5.0,stimulus_colour,255 0 0
39144,ANTI_SACCADE,96,5.0,stimulus_x,624.0
39145,ANTI_SACCADE,96,5.0,stimulus_y,540.0
39146,ANTI_SACCADE,96,5.0,stimulus_time,707.828201


In [ ]:


def get_reaction_time_feature(df: pd.DataFrame) -> pd.DataFrame:
     """_summary_

     Args:
         df (pd.DataFrame): _description_

     Returns:
         pd.DataFrame: _description_
     """
     feature_df = (df
     .query("stimulus_active == True")
     .sort_values(by=["participant_id", "trial_id", "stand_time"])
     .assign(stimulus_time = lambda x: np.select([x.event == "FIXPOINT", x.event != "FIXPOINT"], [x.stand_time, None]))
     .assign(stimulus_time = lambda x: x["stimulus_time"].ffill())
     .assign(saccade_direction = lambda x: np.where(x["sacc_end_x"] > x["sacc_start_x"], "right", "left"))
     .assign(is_saccade_direction_correct = lambda x: np.where(x["saccade_direction"] != x["stimulus_side"], True, False))
     .query("event == 'ESACC'")
     .groupby(["experiment","participant_id", "trial_id", "is_saccade_direction_correct"])
     .first()
     .reset_index()
     .assign(reaction_time = lambda x: x["stand_start_time"] - x["stimulus_time"])
     .groupby(["experiment","participant_id","is_saccade_direction_correct"])
     .agg(mean_reaction_time = ('reaction_time', 'mean'))
     .reset_index()
     .pivot(index=["experiment", "participant_id"], columns="is_saccade_direction_correct",values="mean_reaction_time")
     .reset_index()
     .rename({True: 'correct_reaction_time',
               False: 'incorrect_reaction_time'}, axis=1)
     )

     return feature_df

def get_pre_calculated_metrics_feature(df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns pd.Dataframe with columns ['experiment','participant_id', X_FEATURES],
    where X_FEATURES is a collection of features found by the following cartesian product:
    {'peak_velocity', 'amplitude', 'duration', 'avg_pupil_size'} x {np.mean, np.min, np.max, np.median, np.std}
    """
    features_df = (df.groupby(["experiment", "participant_id"])
    .agg({'peak_velocity': [np.mean, np.min, np.max, np.median, np.std],
        'amplitude': [np.mean, np.min, np.max, np.median, np.std],
        'duration': [np.mean, np.min, np.max, np.median, np.std],
        'avg_pupil_size': [np.mean, np.min, np.max, np.median, np.std]
        })
    .reset_index()
    .pipe(rename_columns))
    
    return features_df




In [1]:
get_reaction_time_feature(df_event)

NameError: name 'get_reaction_time_feature' is not defined

# Sample features

In [ ]:
def get_acceleration_feature(df: pd.DataFrame) -> pd.DataFrame:
    """Finds acceleration features for anti saccade experiment

    Args:
        df (pd.DataFrame): Dataframe with raw samples

    Returns:
        pd.DataFrame: Dataframe with columns ['experiment','participant_id', X_FEATURES]
        where X_FEATURES is a collection of features found by the following cartesian product:
        {'total_acceleration_magnitude_left', 'total_acceleration_magnitude_right'} x {np.mean, np.min, np.max, np.median, np.std}
    """

    acceleration = (df.join((df
    .groupby(["experiment", "participant_id", "trial_id"])[['x_velocity_left', 'y_velocity_left', 'x_velocity_right', 'y_velocity_right']].shift(1)
    .rename(columns={'x_velocity_left': 'x_velocity_left_lagged'
            , 'y_velocity_left': 'y_velocity_left_lagged'
            , 'x_velocity_right': 'x_velocity_right_lagged'
            , 'y_velocity_right': 'y_velocity_right_lagged'}))
    ).assign(x_acceleration_left = lambda x: (x["x_velocity_left"] - x["x_velocity_left_lagged"]) / (1/2000),
            y_acceleration_left = lambda x: (x["y_velocity_left"] - x["y_velocity_left_lagged"]) / (1/2000),
            x_acceleration_right = lambda x: (x["x_velocity_right"] - x["x_velocity_right_lagged"]) / (1/2000),
            y_acceleration_right = lambda x: (x["y_velocity_right"] - x["y_velocity_right_lagged"]) / (1/2000))
    .assign(total_acceleration_magnitude_left = lambda x: np.sqrt( np.power(x["x_acceleration_left"], 2) + np.power(x["y_acceleration_left"], 2)),
            total_acceleration_magnitude_right = lambda x: np.sqrt( np.power(x["x_acceleration_right"], 2) + np.power(x["y_acceleration_right"], 2)))
    .groupby(["experiment", "participant_id", "trial_id"])
    .agg({'total_acceleration_magnitude_left': [np.mean, np.min, np.max, np.median, np.std],
        'total_acceleration_magnitude_right': [np.mean, np.min, np.max, np.median, np.std]
        })
    .reset_index()
    .pipe(rename_columns)
    )


# Get all features

In [ ]:
def get_anti_saccade_features(df_event: pd.DataFrame, df_sample:pd.DataFrame) -> pd.DataFrame:
    """Runs all anti saccade features extractions

    Args:
        df (pd.DataFrame): The preprocessed dataframe

    Returns:
        pd.DataFrame: Dataframe with columns ["experiment", "participant_id", X_FEATURES], where X_FEATURES is a collection of features
    """
    
    event_feature_functions = [get_pre_calculated_metrics_feature, get_n_correct_trials_feature, get_prop_trials_feature, get_reaction_time_feature]
    df_event_features_list = [f(df=df_event) for f in event_feature_functions]
    
    sample_feature_functions = [get_acceleration_feature]
    df_sample_features_list = [f(df=df_sample) for f in sample_feature_functions]
    
    df_features_list = df_event_features_list + df_sample_features_list
    
    df_features = reduce(lambda x, y: pd.merge(x, y, on = ["experiment", "participant_id"]), df_features_list)
    
    return df_features


features = get_anti_saccade_features(df_event=df_event, df_sample=df_sample)
    
    

/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_26493/232577473.py:8: FutureWarning: The provided callable <function mean at 0x10772bba0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  .agg({'peak_velocity': [np.mean, np.min, np.max, np.median, np.std],
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_26493/232577473.py:8: FutureWarning: The provided callable <function min at 0x10772b2e0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg({'peak_velocity': [np.mean, np.min, np.max, np.median, np.std],
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_26493/232577473.py:8: FutureWarning: The provided callable <function max at 0x10772b1a0> is currently using SeriesGroupBy.max. In a future version of pandas, th

UndefinedVariableError: name 'stimulus_active' is not defined

# Join demographic info on features

In [ ]:
def load_demographic_info() -> pd.DataFrame:
    demographics = pd.read_excel(DATA_DIR / "demographic_info.xlsx")[["ID", "Group"]]

    demographics["y"] = (demographics["Group"] == "PATIENT").astype(int)
    demographics["participant_id"] = demographics["ID"].astype(str)
    demographics = demographics[["participant_id", "y"]]
    return demographics


def join_demographic_info_on_features(feature_df: pd.DataFrame) -> pd.DataFrame:
    demographics = load_demographic_info()
    return pd.merge(feature_df, demographics, how='left', on='participant_id')
    
    
join_demographic_info_on_features(feature_df=features)

# Eye disconjugacy
Paper: https://www.liebertpub.com/doi/full/10.1089/neu.2014.3687

In [ ]:
data = (pd.read_csv("../data/reading_eyelink.asc", delimiter="\t", names={'time':int, 'x_left':float, 'y_left':float, 'd1':str, 'x_right':float, 'y_right':float, 'd2':str, 'd3':str, 'd4':str}) # sample data from internet
 .assign(experiment = lambda x: "ANTI_SACCADE")
)

data=(data
#    .sort_values(["experiment", "participant_id", "trial_id", "time"])
    .groupby(["experiment", "participant_id"])
    .apply(lambda group: group.assign(
        x_left_rolling=group["x_left"].rolling(window=5, min_periods=1).mean().reset_index(drop=True),
        x_right_rolling=group["x_right"].rolling(window=5, min_periods=1).mean().reset_index(drop=True),
        y_left_rolling=group["y_left"].rolling(window=5, min_periods=1).mean().reset_index(drop=True),
        y_right_rolling=group["y_right"].rolling(window=5, min_periods=1).mean().reset_index(drop=True),
    ))
    .reset_index(drop=True)
    .assign(
        X_diffs = lambda x: ((x["x_left_rolling"] - x["x_right_rolling"]) - 0)**2,
        Y_diffs = lambda x: ((x["y_left_rolling"] - x["y_right_rolling"]) - 0)**2
    )
    .groupby(["experiment", "participant_id"])
    .apply(lambda group: group.assign(
        X_squared_scaled = group["X_diffs"] / group.shape[0],
        Y_squared_scaled = group["Y_diffs"] / group.shape[0]
    ))
    .reset_index(drop=True)
    .groupby(["experiment", "participant_id"])
    .agg(
        Var_X = ("X_squared_scaled", "sum"),
        Var_Y = ("Y_squared_scaled", "sum")
    )
    .assign(
        Var_total = lambda x: x["Var_X"] + x["Var_Y"]
    )
    .reset_index()
)


data

,experiment,Var_X,Var_Y,Var_total
0,ANTI_SACCADE,148.074193,72.733417,220.80761
